# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [5]:

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        frame = cv2.resize(frame, (900, 700))
        
      
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
       
        results = pose.process(image)
    
       
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
mp_drawing.DrawingSpec

mediapipe.python.solutions.drawing_utils.DrawingSpec

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [6]:

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        frame = cv2.resize(frame, (900, 700))
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.6269943
y: 0.511625
z: -1.2530736
visibility: 0.9994561
, x: 0.65686214
y: 0.43432844
z: -1.2165488
visibility: 0.99881244
, x: 0.6772011
y: 0.4343651
z: -1.2164359
visibility: 0.99883693
, x: 0.6969718
y: 0.43566534
z: -1.2168753
visibility: 0.9985702
, x: 0.5982571
y: 0.4335749
z: -1.2309101
visibility: 0.99902046
, x: 0.5755559
y: 0.43291932
z: -1.2305087
visibility: 0.99908555
, x: 0.55146974
y: 0.4332197
z: -1.2307068
visibility: 0.99906
, x: 0.7138692
y: 0.46123224
z: -0.83699757
visibility: 0.99871624
, x: 0.49534228
y: 0.45529956
z: -0.88163745
visibility: 0.9996275
, x: 0.6560181
y: 0.58409756
z: -1.0950549
visibility: 0.9996543
, x: 0.5885631
y: 0.57785785
z: -1.1071404
visibility: 0.9998449
, x: 0.80687773
y: 0.7499197
z: -0.5361031
visibility: 0.99847764
, x: 0.33006424
y: 0.74547195
z: -0.60609514
visibility: 0.9990595
, x: 0.82227564
y: 1.175158
z: -0.40222612
visibility: 0.12693258
, x: 0.20877132
y: 1.0573493
z: -0.47812307
visibility: 0.5216431
, x: 0.77234066
y:

In [7]:
len(landmarks)

33

In [8]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [9]:
landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].visibility

0.4811214506626129

In [10]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.82688576
y: 0.99350584
z: -0.4987881
visibility: 0.6552075

In [11]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.8634801
y: 1.3289198
z: -0.98852724
visibility: 0.52361256

# 3. Calculate Angles

In [12]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [13]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [14]:
shoulder, elbow, wrist

([0.7449470162391663, 0.6678153276443481],
 [0.8268857598304749, 0.9935058355331421],
 [0.8634800910949707, 1.328919768333435])

In [15]:
calculate_angle(shoulder, elbow, wrist)

172.10479076640573

In [16]:
tuple(np.multiply(elbow, [640, 480]).astype(int))

(529, 476)

In [ ]:
# # Initialize Mediapipe
# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Video clip input
# video_path = 'D:\Downloads\IMG_0601.MOV'  # Change this to your video file path

# cap = cv2.VideoCapture(video_path)
# ## Setup mediapipe instance
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()

#         frame = cv2.resize(frame, (900, 700))
        
#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
      
#         # Make detection
#         results = pose.process(image)
    
#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
#         # Extract landmarks
#         try:
#             landmarks = results.pose_landmarks.landmark
            
#             # Get coordinates
#             shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
#             elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
#             wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
#             # Calculate angle
#             angle = calculate_angle(shoulder, elbow, wrist)
            
#             # Visualize angle
#             cv2.putText(image, str(angle), 
#                            tuple(np.multiply(elbow, [680, 420]).astype(int)), 
#                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
#                                 )
                       
#         except:
#             pass
        
        
#         # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

In [17]:
# Initialize Mediapipe
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            # Release video capture and close windows
            cap.release()
            cv2.destroyAllWindows()
            break

        frame = cv2.resize(frame, (900, 700))

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
                
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)

            # Visualize angle
            cv2.putText(image, f"Angle: {angle:.2f}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                   mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                   mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                   )               

        cv2.imshow('Mediapipe Feed', image)

            # Wait for 'q' key to stop the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
                